## Monday Meeting 


I have adapted the MLP dataprovider to the accelerometer data set to be used. I found the NN coded by Pawel extremely well documented and useful and thus I have picked it over Lasagne, tensorflow or the horrendous scikit learn (on which I wasted a whole day pointlessly).


In [ ]:
import numpy
import logging
from mlp.dataset import *

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = ACLDataProvider(dset='train', batch_size=100, max_num_batches=-10, randomize=True)
valid_dp = ACLDataProvider(dset='valid', batch_size=1824, max_num_batches=1, randomize=False)

In [ ]:
#Baseline experiment
from copy import deepcopy
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateFixed

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 100
learning_rate = 0.5
max_epochs = 30
cost = CECost()
    
stats = list()

test_dp = deepcopy(valid_dp)
train_dp.reset()
valid_dp.reset()
test_dp.reset()

#define the model
model = MLP(cost=cost)
model.add_layer(Sigmoid(idim=125, odim=125*2, irange=1.6, rng=rng))
model.add_layer(Softmax(idim=125*2, odim=19, rng=rng))

# define the optimiser, here stochasitc gradient descent
# with fixed learning rate and max_epochs
lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

logger.info('Training started...')
tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

logger.info('Testing the model on test set:')

tst_cost, tst_accuracy = optimiser.validate(model,test_dp )
logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))



In [1]:
import numpy
import logging
from mlp.dataset import *

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = ACLDataProvider(dset='train', batch_size=100, max_num_batches=-10, randomize=True)
valid_dp = ACLDataProvider(dset='valid', batch_size=1824, max_num_batches=1, randomize=False)


#Baseline experiment
from copy import deepcopy
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.layers import * 
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateFixed

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 100
learning_rate = 0.5
max_epochs = 30
cost = CECost()
    
stats = list()

test_dp = deepcopy(valid_dp)
train_dp.reset()
valid_dp.reset()
test_dp.reset()

#define the model
model = MLP(cost=cost)
model.add_layer(ComplexLinear(idim=125, odim=125, irange=1.6, rng=rng))
model.add_layer(Sigmoid(idim=2*125, odim=125, irange=1.6, rng=rng))
model.add_layer(Softmax(idim=125, odim=19, rng=rng))

# define the optimiser, here stochasitc gradient descent
# with fixed learning rate and max_epochs
lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

logger.info('Training started...')
tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

logger.info('Testing the model on test set:')

tst_cost, tst_accuracy = optimiser.validate(model,test_dp )
logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))



INFO:root:Initialising data providers...


(7296, 500) (7296,)
(7296, 125)
(1824, 500)

INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.988. Accuracy is 4.85%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.980. Accuracy is 4.61%
mlp/optimisers.py:186: ComplexWarning: Casting complex values to real discards the imaginary part
  % (self.lr_scheduler.epoch + 1, cost_name, tr_nll, tr_acc * 100.))
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 2.984. Accuracy is 12.96%
mlp/optimisers.py:194: ComplexWarning: Casting complex values to real discards the imaginary part
  % (self.lr_scheduler.epoch + 1, cost_name, valid_nll, valid_acc * 100.))
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 2.884. Accuracy is 12.72%
INFO:mlp.optimisers:Epoch 1: Took 1 seconds. Training speed 13587 pps. Validation speed 45625 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 2.991. Accuracy is 12.93%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 2.798. Accuracy is 14.42%
INFO:mlp.optimisers:Epoch 2: 

 (1824,)
(1824, 125)


In [ ]:
import numpy
import logging
from mlp.dataset import *

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = ACLDataProvider(dset='train', batch_size=100, max_num_batches=-10, randomize=True)
valid_dp = ACLDataProvider(dset='valid', batch_size=1824, max_num_batches=1, randomize=False)


#Baseline experiment
from copy import deepcopy
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.layers import * 
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateFixed

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 100
learning_rate = 0.07
max_epochs = 30
cost = CECost()
    
stats = list()

test_dp = deepcopy(valid_dp)
train_dp.reset()
valid_dp.reset()
test_dp.reset()


#define the model
model = MLP(cost=cost)
model.add_layer(DFTLinear(idim=125, odim=125, irange=1.6, rng=rng))
#Every activation function from dft layer produces two values (x,y) for x+iy
model.add_layer(Sigmoid(idim=125*2, odim=125*2, irange=1.6, rng=rng))
model.add_layer(Softmax(idim=125*2, odim=19, rng=rng))

# define the optimiser, here stochasitc gradient descent
# with fixed learning rate and max_epochs
lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

logger.info('Training started...')
tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

logger.info('Testing the model on test set:')

tst_cost, tst_accuracy = optimiser.validate(model,test_dp )
logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))